In [0]:
filename = dbutils.widgets.get(`filename`)
fnamewithoutext = filename.split('.')[0]
print(filename)

In [0]:
dbutils.fs.unmount('/mnt/sales')

In [0]:
alreadyMounted = False
for x in dbutils.fs.mounts():
    if x.mountPoint == "/mnt/sales" :
      alreadyMounted = True
      break
    else:
        alreadyMounted = False
print(alreadyMounted)


In [0]:
if not alreadyMounted:
  dbutils.fs.mount(source = 'wasbs://sales@trendytechsaproject.blob.core.windows.net', mount_point = '/mnt/sales',
extra_configs = {'fs.azure.account.key.trendytechsaproject.blob.core.windows.net': 'Z5+E5DIhGSQeFUWzv8pHoYaWEy+V+NimF7oGG9EKJziRX36InbYLvfBBz9+aE/SJ0jZG7VYsiJY9+ASty6biOA=='})
  alreadyMounted = True
  print("Mounting done successfully")
else:
  print("It is already mounted")

In [0]:
dbutils.fs.unmount('/mnt/sales')
dbutils.fs.mount(source = 'wasbs://sales@trendytechsaproject.blob.core.windows.net', mount_point = '/mnt/sales',
extra_configs = {'fs.azure.account.key.trendytechsaproject.blob.core.windows.net': storageAccountKey})

/mnt/sales has been unmounted.
Out[2]: True

In [0]:
storageAccountKey = dbutils.secrets.get(scope = 'databricksScope', key = 'storage-account-key')

In [0]:
%fs
ls  /mnt/sales

path,name,size,modificationTime
dbfs:/mnt/sales/discarded/,discarded/,0,0
dbfs:/mnt/sales/landing/,landing/,0,0
dbfs:/mnt/sales/staging/,staging/,0,0


In [0]:
%fs 
ls /mnt/sales/landing

path,name,size,modificationTime
dbfs:/mnt/sales/landing/orders.csv,orders.csv,258,1761161823000


In [0]:
ordersDF = spark.read.csv('/mnt/sales/landing/{}'.format(filename)', inferSchema= True, header = True)

In [0]:
display(ordersDF)

order_id,order_date,customer_id,order_status
1111111,2013-07-25T00:00:00.000+0000,11599,CLOSED
2222222,2013-07-25T00:00:00.000+0000,256,PENDING_PAYMENT
3333333,2013-07-25T00:00:00.000+0000,12111,COMPLETE
4444444,2013-07-25T00:00:00.000+0000,8827,CLOSED


In [0]:
errorFlg = False
ordersCount = ordersDF.count()
print(ordersCount)

distinctordersCount = ordersDF.select('order_id').distinct().count()
print(distinctordersCount)

if ordersCount != distinctordersCount:
    errorFlg = True
if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename), '/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg": "true", "errorMsg": "Orderid is repeated"}')
else:
    ordersDF.createOrReplaceTempView("orders")

4
4


In [0]:
dbServer:'trendytechsqlserverprojectt'
dbPort :1433
dbName : 'trendytechsqldbproject'
dbUser : 'ttsqluser'
dbPassword : 'Pooja1234'
databricksScope : 'saleprojectscopenew'


In [0]:

connectionUrl = 'jdbc:sqlserver://{}.database.windows.net:{}; database={}; user={};'.format(dbServer, dbPort, dbName, dbUser)


connectionProperties = {
      'password' : 'Pooja1234',
      'driver': 'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}

In [0]:
ValidstatusDF = spark.read.jdbc(url = connectionUrl, table = 'dbo.valid_order_status', properties = connectionProperties)

In [0]:
dbutils.secrets.listScopes()

Out[13]: [SecretScope(name='salesprojectsccope')]

In [0]:
display(ValidstatusDF)

status_name
ON_HOLD
PAYMENT_REVIEW
PROCESSING
CLOSED
SUSPECTED_FRAUD
COMPLETE
PENDING
CANCELLED
PENDING_PAYMENT


In [0]:
ValidstatusDF.createOrReplaceTempView("valid_status")

In [0]:
invalid_rows_df = spark.sql("select * from orders where order_status not in (select * from valid_status)")
display(invalid_rows_df)

order_id,order_date,customer_id,order_status


In [0]:
if invalid_rows_df.count() >0:
    errorFlg = True
if errorFlg:
    dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/discarded')
    dbutils.notebook.exit('{"errorFlg": "true", "errorMsg": "Invalid order status found"}')
else:
     dbutils.fs.mv('/mnt/sales/landing/{}'.format(filename),'/mnt/sales/staging')
     dbutils.notebook.exit('{"errorFlg": "true", "errorMsg": "All good"}')

In [0]:
orderItemsDf = spark.read.csv('/mnt/sales/order_items/order_items.csv', InferSchema = True, header = True)
display(orderItemsDf)
orderItemsDf.createOrReplaceTempView("order_items")

In [0]:
customerDF = spark.read.jdbc(url = connectionUrl, table = 'dbo.customers', properties = connectionProperties)
display(customerDF)
customerDF.createOrReplaceTempView("customers")

In [0]:
resultdf = spark.sql("select customers.customer_id, customers.customers_fname, customers.customers_lname, customers.customer_city, customers.customer_state, customers.customer_zipcode, count(order_id) as num_of_orders_placed, sum(order_item_sub_total),2) AS total_amount from customers, orders, order_items where customers.customer_id = orders.customer_id and orders.order_item_id = order_items.order_item_order_id group by customers.customer_id, customers.customers_fname, customers.customers_lname, customers.customer_city, customers.customer_state, customers.customer_zipcode order by total_amount desc")


In [0]:
display(resultdf)
resultdf.write.jdbc(url = connectionUrl, table = 'dbo.sales_reporting', properties = connectionProperties, mode = 'overwrite')